In [15]:
import pandas as pd
import math

#pull reviews
reviews = pandas.read_csv("reviews.csv")

#take train data...
num_test = math.floor(len(reviews) * .2)
num_train = len(reviews) - num_test

print(len(reviews))
print(num_test)
print(num_train)

2511
502
2009


In [16]:
test = reviews.iloc[:num_test]
train = reviews.iloc[num_test:]

print(len(test))
print(len(train))

502
2009


In [29]:
test_labels = test['rating'].to_frame()
test_text = test['text'].to_frame()

train_labels = train['rating'].to_frame()
train_text = train['text'].to_frame()

In [35]:
import nltk
from nltk.tokenize import WordPunctTokenizer

tk = WordPunctTokenizer()
NEG = 'NOT_'
negs = ['not', 'none', 'no', 'impossible', 'n\'t']
punct = [',', '.', '?', '!', ';', '-']

#preps data
def prep(text):
    data_list = []
    for i, data in text.iterrows():
        words = []
        review = data['text']
        listed = review.split()
        neggy = False
        for word in listed:
            w = word.lower()
            if neggy:
                if word in punct:
                    words.append(w)
                    neggy = False
                else:
                    words.append(NEG + w)
            else:
                words.append(w)
                if w in negs:
                    if neggy:
                        neggy = False
                    else:
                        neggy = True
        data_list.append(' '.join(words))
    return data_list
 
train_list = prep(train_text)
tokens = tk.tokenize(' '.join(train_list))
freq = nltk.FreqDist(tokens)
vocab = {'<unk>':0}
i = 1
for item in freq.keys():
    if item not in vocab:
        if freq[item] > 2:
            vocab[item] = i
            i += 1
print(len(vocab))

11768


In [37]:
import numpy as np

def make_matrix(text_list):
    n = len(text_list)
    X = np.zeros((n, len(vocab)))
    for i in range(n):
        review = tk.tokenize(text_list[i])
        for word in review:
            if word in vocab:
                X[i][vocab[word]] += 1
            else:
                X[i][0] += 1
    return X

train_matrix = make_matrix(train_list)

print("train feature matrix shape: ({}, {})".format(len(train_matrix), len(train_matrix[0])))
print("train label array shape: ({},)".format(len(train_labels)))

train feature matrix shape: (2009, 11768)
train label array shape: (2009,)


In [39]:
#clean test data

test_list = prep(test_text)
test_matrix = make_matrix(test_list)

print("test feature matrix shape: ({}, {})".format(len(test_matrix), len(test_matrix[0])))
print("test label array shape: ({},)".format(len(test_labels)))

test feature matrix shape: (502, 11768)
test label array shape: (502,)
